In [2]:
import os
import pandas as pd
import numpy as np

In [4]:
hfscatfilesraw = os.listdir('catalog_hfs')
hfscatfilesraw.sort()
hfscatfiles = [x for x in hfscatfilesraw if "backup" not in x]
hfscatcodes = []
for hfscatfile in hfscatfiles:
    hfscatcodes.append(hfscatfile.split("_")[0].replace("c00","").replace("c0","").replace("c",""))
len(hfscatcodes)

204

In [6]:
hfsdict = dict(zip(hfscatcodes,hfscatfiles))
hfsdict

{'14502': 'c014502_hfs.cat',
 '17506-7QN': 'c017506-7QN_hfs.cat',
 '17506': 'c017506_hfs.cat',
 '19501': 'c019501_hfs.cat',
 '27501': 'c027501_hfs.cat',
 '27503': 'c027503_hfs.cat',
 '27507': 'c027507_hfs.cat',
 '28501': 'c028501_hfs.cat',
 '28504': 'c028504_hfs.cat',
 '28509': 'c028509_hfs.cat',
 '29501': 'c029501_hfs.cat',
 '29506': 'c029506_hfs.cat',
 '29510': 'c029510_hfs.cat',
 '29518': 'c029518_hfs.cat',
 '30501': 'c030501_hfs.cat',
 '30504': 'c030504_hfs.cat',
 '30505': 'c030505_hfs.cat',
 '30510': 'c030510_hfs.cat',
 '30512': 'c030512_hfs.cat',
 '30514': 'c030514_hfs.cat',
 '30515': 'c030515_hfs.cat',
 '31514': 'c031514_hfs.cat',
 '31517': 'c031517_hfs.cat',
 '32503': 'c032503_hfs.cat',
 '32505': 'c032505_hfs.cat',
 '39501': 'c039501_hfs.cat',
 '39508': 'c039508_hfs.cat',
 '39509': 'c039509_hfs.cat',
 '39510': 'c039510_hfs.cat',
 '40514': 'c040514_hfs.cat',
 '40516': 'c040516_hfs.cat',
 '40517': 'c040517_hfs.cat',
 '40518': 'c040518_hfs.cat',
 '41503': 'c041503_hfs.cat',
 '4150

In [7]:
# CHECK REPEATED
seen = set()
uniq = []
print("REPEATED ENTRIES")
for x in hfscatcodes:
    if x not in seen:
        uniq.append(x)
        seen.add(x)
    else:
        print(x)
        matching = [s for s in hfscatfiles if x in s]
        print (matching)        

REPEATED ENTRIES


In [8]:
fpartition = open('partition_function.html',"r")
fpartitionlines = fpartition.readlines()
fpartition.close()

In [11]:
def calculateCorrectionFactor(fpartitioncode):
    threshold=0.02
    molec = pd.read_fwf('catalog/%s' % hfsdict[fpartitioncode].replace('_hfs',''),header = None,colspecs=[(0,13),(21,29)])
    molechfs = pd.read_fwf('catalog_hfs/%s' % hfsdict[fpartitioncode],header = None,colspecs=[(0,13),(21,29)])
    reffreq = molechfs.iloc[1][0] # Not the first hiperfine line
    print ("Representative frequency: %s" %reffreq)
    moleclineshfs = np.array(molechfs[(molechfs[0]>reffreq*(1-threshold)) & (molechfs[0]<reffreq*(1+threshold))][1])
    moleclineshfssum = np.power(10,moleclineshfs).sum()
    print ("Sum HFS: %s" %moleclineshfssum)
    moleclines = np.array(molec[(molec[0]>reffreq*(1-threshold)) & (molec[0]<reffreq*(1+threshold))][1])
    moleclinessum = np.power(10,moleclines).sum()
    print ("Sum : %s" %moleclinessum)
    print ("MOLEC/MOLECHFS: %s "%(moleclinessum/moleclineshfssum))
    print("----------FACTOR: %s" %int(round(moleclinessum/moleclineshfssum)))
    return int(round(moleclinessum/moleclineshfssum))

In [12]:
calculateCorrectionFactor(124507)

KeyError: 124507

In [13]:
fpartitionoutput = open('partition_function_hfs.html',"w")
hfsfoundcodes = []
for fpartitionline in fpartitionlines:
    fpartitioncode = fpartitionline[0:6].replace(" ","")
    if fpartitioncode in hfscatcodes:
        hfsfoundcodes.append(fpartitioncode)
        print("%s---------%s" %(fpartitionline[7:33],fpartitioncode))
        #factor = calculateCorrectionFactor(fpartitioncode)
        auxmolname =fpartitionline[7:33].rstrip()
        #print(fpartitionline[7:33].replace("%s    "%auxmolname,"%s,hfs"%auxmolname)+"---")
        fpartitionoutput.write(fpartitionline.replace("%s    "%auxmolname,"%s,hfs"%auxmolname))
fpartitionoutput.close()

print("")
print("FOUND %s out of %s" %(len(hfsfoundcodes), len(hfscatcodes)))
print("NOT FOUND CODES (consider repeated above)")
list (set(hfscatcodes)-set(hfsfoundcodes))

C-13-H+                   ---------14502
NH3-wHFS                  ---------17506
NHD2                      ---------19501
HCN, v=0                  ---------27501
HCN, v2=1                 ---------27503
HCN, v2=2                 ---------27507
HC-13-N, v=0              ---------28501
HCNH+                     ---------28504
DCN, v=0                  ---------28509
C-13-O                    ---------29501
N2H+, v=0; recommended    ---------29506
DC-13-N                   ---------29510
H2CNH                     ---------29518
H2CO                      ---------30501
HC-13-O+                  ---------30504
HCO-17+                   ---------30505
DCO+                      ---------30510
NO+, v=0,1                ---------30512
H2C-13-NH                 ---------30514
H2CND                     ---------30515
H2CO-17                   ---------31514
DCO-17+                   ---------31517
H2CO-18                   ---------32503
DCO-18+                   ---------32505
HCCN            

['17506-7QN', '75518LTE']

In [14]:
calculateCorrectionFactor('28501')

Representative frequency: 86340.163
Sum HFS: 0.0028349936432389084
Sum : 0.0028255305024595567
MOLEC/MOLECHFS: 0.9966620239865722 
----------FACTOR: 1


1

In [ ]:
hfscatcodes